In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
torch.set_grad_enabled(False)
from lavis.common.registry import registry
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm
import gc

/opt/conda/envs/eval/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/opt/conda/envs/eval/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [6]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration, AutoProcessor
import torch


BLIP2DICT = {
    "FlanT5 XXL": "Salesforce/blip2-flan-t5-xxl",
    "FlanT5 XL COCO": "Salesforce/blip2-flan-t5-xl-coco",
    "OPT6.7B COCO": "Salesforce/blip2-opt-6.7b-coco",
    "OPT2.7B COCO": "Salesforce/blip2-opt-2.7b-coco",
    "FlanT5 XL": "Salesforce/blip2-flan-t5-xl",
    "OPT6.7B": "Salesforce/blip2-opt-6.7b",
    "OPT2.7B": "Salesforce/blip2-opt-2.7b",
}


class Blip2:
    def __init__(self, model, device_id):
        # load BLIP-2 to a single gpu
        self.tag = model
        self.device = "cuda:{}".format(device_id)

        dtype = {"torch_dtype": torch.float16}#{"load_in_8bit": True} if self.bit8 else {"torch_dtype": torch.float16}
        self.blip2_processor = AutoProcessor.from_pretrained(BLIP2DICT[self.tag])
        self.blip2 = Blip2ForConditionalGeneration.from_pretrained(
            BLIP2DICT[self.tag], device_map={"": device_id}, torch_dtype=torch.float16
        )
        # self.blip2 = Blip2ForConditionalGeneration.from_pretrained(BLIP2DICT[self.tag], device_map='auto', load_in_8bit=True)

    def ask(self, raw_image, question):
        inputs = self.blip2_processor(raw_image, question, return_tensors="pt").to(self.device, torch.float16)
        out = self.blip2.generate(**inputs)
        answer = self.blip2_processor.decode(out[0], skip_special_tokens=True)
        return answer

    def nucleus_ask(self, raw_image, question):
        inputs = self.blip2_processor(raw_image, question, return_tensors="pt").to(self.device, torch.float16)
        # print(self.blip2)
        print(inputs)
        generated_ids = self.blip2.generate(
            **inputs,
            # pixel_values=inputs.pixel_values,
            do_sample="Nucleus sampling",
            temperature=0.5,
            length_penalty=-1.0,
            repetition_penalty=1.0,
            max_length=30,
            min_length=1,
            num_beams=5,
            top_p=0.1
        )
        print(generated_ids)
        result = self.blip2_processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        return result

    def caption(self, raw_image, mode="default"):
        if mode == "default":
            # starndard way to caption an image in the blip2 paper
            caption = self.ask(raw_image, "a photo of")
            caption = caption.replace("\n", " ").strip()  # trim caption
        elif mode == "nucleus":
            caption = self.nucleus_ask(raw_image, "a photo of")
        return caption

In [7]:
bt2 = Blip2("FlanT5 XL", 0)

import pickle
with open('outputs.pkl', 'rb') as f:
    outputs = pickle.load(f)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/envs/eval/lib/python3.8/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


robots playing a game of chess


In [10]:
answer = bt2.ask(outputs['bin'], 'What is this a photo of?')
print(answer)

a wooden table with a knife and a knife


In [2]:
ckpt_path = "checkpoints/pretrain_blip2_sam_flant5xl_v2.pth"
model_cfg = {
    "arch": "blip2_t5",
    "model_type": "pretrain_flant5xl",
    "use_grad_checkpoint": False,
}
model_cfg = OmegaConf.create(model_cfg)
model = registry.get_model_class("blip2_t5").from_pretrained(model_type="pretrain_flant5xl")
checkpoint = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(checkpoint["model"], strict=False)
model.eval()

checkpoint = None
del checkpoint
gc.collect()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

174

In [3]:
text_tokens = model.t5_tokenizer("banana", padding="longest", return_tensors="pt")
input_ids = text_tokens.input_ids
text_features = model.t5_model.encoder.embed_tokens(input_ids)

model.t5_model = None
gc.collect()

0

In [4]:
text_output = model.Qformer.bert(
            text_tokens.input_ids,
            attention_mask=text_tokens.attention_mask,
            return_dict=True,
        )
print(text_output.last_hidden_state.shape)

TypeError: 'NoneType' object is not callable

In [6]:
print(model.Qformer.embeddings)

AttributeError: 'BertLMHeadModel' object has no attribute 'embeddings'

In [4]:
print(text_features.shape)

torch.Size([1, 2, 2048])


30

In [7]:
pc_embeds = torch.load('pixelwise_features.pt')
pc = torch.load('xyz_pts.pt')
all_pcs = torch.zeros((pc_embeds.shape), device='cpu')
for j in range(pc.shape[0]):
    pcs = []
    for i in range(3):
        pc_i = pc[j][:, i]
        pcs.append(model.pos_embedding[pc_i].cpu())
    pcs = torch.cat(pcs, -1).to('cpu')
    all_pcs[j][:, :1407] = pcs

pc = None
del pc
gc.collect()

149

In [8]:
pc_embeds += 0.01 * all_pcs

all_pcs = None
del all_pcs
gc.collect()

30

In [9]:
print(pc_embeds.shape)

torch.Size([1, 1041540, 1408])


In [9]:
image_atts = torch.ones(pc_embeds.size()[:-1], dtype=torch.long)
query_tokens = model.query_tokens.expand(pc_embeds.shape[0], -1, -1)
query_output = model.Qformer.bert(
    query_embeds=query_tokens,
    encoder_hidden_states=pc_embeds.float(),
    encoder_attention_mask=image_atts,
    return_dict=True,
)

query_tokens = None
pc_embeds = None
image_atts = None
del query_tokens
del pc_embeds
del image_atts
gc.collect()

30

In [10]:
print(query_output.last_hidden_state.shape)

torch.Size([1, 32, 768])


In [8]:
pointcloud_features = model.t5_proj(query_output.last_hidden_state)

query_output = None
del query_output
gc.collect()

129

In [9]:
print(pointcloud_features.shape)

torch.Size([1, 32, 2048])


AttributeError: 'NoneType' object has no attribute 'encoder'

In [ ]:
temp = model.temp

model = None
del model
gc.collect()

In [ ]:
from lavis.models.base_model import concat_all_gather
pointcloud_feats_all = concat_all_gather(pointcloud_features)

sim_t2q = torch.matmul(text_features.unsqueeze(1).unsqueeze(1), pointcloud_feats_all.permute(0, 2, 1)).squeeze()

# text-image similarity: aggregate across all query tokens
sim_t2i, _ = sim_t2q.max(-1)
sim_t2i = sim_t2i / temp